In [12]:
!python -V

/bin/bash: python: command not found


In [13]:
import pandas as pd

%pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org mlflow==3.1.1


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import pickle

In [15]:
%pip install seaborn
import seaborn as sns
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [17]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/ruoke/Downloads/Mlops_Zoomcamp/experiment_tracking/mlruns/1', creation_time=1751294384080, experiment_id='1', last_update_time=1751294384080, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [18]:
def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [19]:
df=read_dataframe('./data/taxi_tripdata.csv')
#get df_train and df_val from df
#by randomly splitting the data into 80% train and 20% validation
df_train = df.sample(frac=0.8, random_state=42)
df_val = df.drop(df_train.index)


/var/folders/40/y3y5nqk55yn7cb9wd9v3z0pr0000gn/T/ipykernel_74087/3970424726.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [20]:
len(df_train), len(df_val)

(63278, 15819)

In [21]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [22]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [23]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [24]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)

71.40238818465828

In [25]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [26]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [27]:
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/Users/ruoke/Downloads/Mlops_Zoomcamp/.venv/lib/python3.11/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

[0]	validation-rmse:12.17166                          
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/ruoke/Downloads/Mlops_Zoomcamp/.venv/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [22:53:01] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[1]	validation-rmse:11.66677                          
[2]	validation-rmse:11.20223                          
[3]	validation-rmse:10.77581                          
[4]	validation-rmse:10.38506                          
[5]	validation-rmse:10.02781                          
[6]	validation-rmse:9.70157                           
[7]	validation-rmse:9.40422                           
[8]	validation-rmse:9.13407                           
[9]	validation-rmse:8.88870                           
[10]	validation-rmse:8.66626                          
[11]	validation-rmse:8.46504                          
[12]	validation-rmse:8.28323                          
[13]	validation-rmse:8.11949                          
[14]	validation-rmse:7.97230                          
[15]	validation-rmse:7.84014                          
[16]	validation-rmse:7.72149                          
[17]	validation-rmse:7.61519                          
[18]	validation-rmse:7.52014                          
[19]	valid

KeyboardInterrupt: 

In [ ]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=300,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/Users/ruoke/Downloads/Mlops_Zoomcamp/.venv/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [22:54:11] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:11.85278
[1]	validation-rmse:11.09323
[2]	validation-rmse:10.43034
[3]	validation-rmse:9.85894
[4]	validation-rmse:9.36512
[5]	validation-rmse:8.93988
[6]	validation-rmse:8.57922
[7]	validation-rmse:8.26839
[8]	validation-rmse:8.00591
[9]	validation-rmse:7.78299
[10]	validation-rmse:7.59600
[11]	validation-rmse:7.43683
[12]	validation-rmse:7.30452
[13]	validation-rmse:7.19303
[14]	validation-rmse:7.10180
[15]	validation-rmse:7.02518
[16]	validation-rmse:6.95948
[17]	validation-rmse:6.90571
[18]	validation-rmse:6.86069
[19]	validation-rmse:6.82202
[20]	validation-rmse:6.78987
[21]	validation-rmse:6.76190
[22]	validation-rmse:6.73973
[23]	validation-rmse:6.71976
[24]	validation-rmse:6.70385
[25]	validation-rmse:6.69016
[26]	validation-rmse:6.67770
[27]	validation-rmse:6.66733
[28]	validation-rmse:6.65707
[29]	validation-rmse:6.64983
[30]	validation-rmse:6.64264
[31]	validation-rmse:6.63693
[32]	validation-rmse:6.63106
[33]	validation-rmse:6.62646
[34]	validation-rmse:

2025/06/30 22:54:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/ruoke/Downloads/Mlops_Zoomcamp/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [22:54:31] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/06/30 22:54:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
